# OCR + PII Extraction Pipeline for Handwritten Documents

## Assignment Objective
Build a simple OCR + PII-extraction pipeline for handwritten documents in JPEG format.

**End-to-end flow**: Input (handwritten JPEG) → Pre-processing → OCR → Text Cleaning → PII Detection → (Optional) Redacted Image

## Features
- ✅ Handles slightly tilted images
- ✅ Works with different handwriting styles
- ✅ Processes doctor/clinic-style notes and forms
- ✅ Hybrid OCR engine (EasyOCR primary with Tesseract fallback)
- ✅ PII detection using regex patterns + spaCy NER
- ✅ Image preprocessing (deskewing, denoising, adaptive thresholding)
- ✅ Optional redaction with bounding boxes

## 1. Import Required Libraries

In [ ]:
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys

# Add the ocr_pipeline module to path
sys.path.append('.')

from ocr_pipeline.preprocess import enhance_for_ocr
from ocr_pipeline.ocr_engine import hybrid_ocr
from ocr_pipeline.text_cleaning import clean_text_from_ocr
from ocr_pipeline.pii_detector import detect_pii
from ocr_pipeline.redactor import redact_image_with_bboxes
from ocr_pipeline.cli import run_pipeline, map_entities_to_bboxes

print("✅ All libraries imported successfully!")

## 2. Pipeline Functions

In [ ]:
def display_image(img_path, title="Image"):
    """Display an image using matplotlib"""
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def process_handwritten_document(img_path, show_steps=True):
    """
    Complete pipeline for processing handwritten documents
    
    Args:
        img_path: Path to input JPEG image
        show_steps: Whether to display intermediate processing steps
    
    Returns:
        dict: Results containing OCR text and PII detections
    """
    print(f"\n🔍 Processing: {img_path}")
    print("=" * 50)
    
    # Step 1: Display original image
    if show_steps:
        display_image(img_path, "Original Handwritten Document")
    
    # Step 2: Preprocessing
    print("📋 Step 1: Preprocessing (deskewing, denoising, thresholding)...")
    proc_path = img_path + ".processed.jpg"
    processed = enhance_for_ocr(img_path, save_path=proc_path)
    
    if show_steps:
        display_image(proc_path, "Preprocessed Image")
    
    # Step 3: OCR - Try original first, then processed
    print("🔤 Step 2: OCR Text Extraction...")
    img_orig = cv2.imread(img_path)
    ocr_results = hybrid_ocr(img_orig)
    
    if not ocr_results:
        print("   Trying processed image...")
        img_proc = cv2.imread(proc_path)
        ocr_results = hybrid_ocr(img_proc)
    
    # Step 4: Text Cleaning
    print("🧹 Step 3: Text Cleaning...")
    full_text = " ".join([clean_text_from_ocr(item['text']) for item in ocr_results])
    print(f"   Extracted Text: {full_text[:100]}{'...' if len(full_text) > 100 else ''}")
    
    # Step 5: PII Detection
    print("🔒 Step 4: PII Detection...")
    pii_entities = detect_pii(full_text)
    
    # Display results
    print(f"\n📊 Results Summary:")
    print(f"   OCR Regions Found: {len(ocr_results)}")
    print(f"   PII Entities Found: {len(pii_entities)}")
    
    if pii_entities:
        print("\n🚨 PII Detected:")
        for i, entity in enumerate(pii_entities, 1):
            print(f"   {i}. {entity['type']}: '{entity['text']}'")
    
    # Step 6: Optional Redaction
    redacted_path = img_path.replace('.jpg', '_redacted.jpg').replace('.jpeg', '_redacted.jpg')
    if pii_entities:
        print(f"\n🖼️  Step 5: Creating redacted image...")
        bboxes = map_entities_to_bboxes(ocr_results, pii_entities)
        if bboxes:
            redact_image_with_bboxes(img_path, bboxes, redacted_path)
            if show_steps:
                display_image(redacted_path, "Redacted Image (PII Blocked)")
    
    return {
        'original_image': img_path,
        'processed_image': proc_path,
        'redacted_image': redacted_path if pii_entities else None,
        'extracted_text': full_text,
        'ocr_results': ocr_results,
        'pii_entities': pii_entities,
        'summary': {
            'ocr_regions': len(ocr_results),
            'pii_count': len(pii_entities),
            'pii_types': list(set([entity['type'] for entity in pii_entities]))
        }
    }

print("✅ Pipeline functions defined!")

## 3. Test with Sample Documents

Let's test our pipeline with the provided sample documents.

In [ ]:
# Test with existing sample
sample_path = "samples/sample1.jpg"

if os.path.exists(sample_path):
    results = process_handwritten_document(sample_path, show_steps=True)
    
    # Save results to JSON
    with open('sample1_results.json', 'w') as f:
        json.dump(results, f, indent=2, default=str)
    
    print(f"\n💾 Results saved to: sample1_results.json")
else:
    print(f"❌ Sample file not found: {sample_path}")
    print("Available files in samples/:")
    if os.path.exists('samples'):
        for f in os.listdir('samples'):
            print(f"   - {f}")

## 4. Process Multiple Documents

Function to process multiple handwritten documents at once.

In [ ]:
def process_multiple_documents(image_paths):
    """
    Process multiple handwritten documents
    
    Args:
        image_paths: List of paths to JPEG images
    
    Returns:
        dict: Combined results from all documents
    """
    all_results = {}
    summary_stats = {
        'total_documents': len(image_paths),
        'total_ocr_regions': 0,
        'total_pii_entities': 0,
        'pii_types_found': set()
    }
    
    for i, img_path in enumerate(image_paths, 1):
        if os.path.exists(img_path):
            print(f"\n📄 Processing Document {i}/{len(image_paths)}")
            results = process_handwritten_document(img_path, show_steps=False)
            
            # Store results
            doc_name = os.path.basename(img_path)
            all_results[doc_name] = results
            
            # Update summary stats
            summary_stats['total_ocr_regions'] += results['summary']['ocr_regions']
            summary_stats['total_pii_entities'] += results['summary']['pii_count']
            summary_stats['pii_types_found'].update(results['summary']['pii_types'])
        else:
            print(f"❌ File not found: {img_path}")
    
    # Convert set to list for JSON serialization
    summary_stats['pii_types_found'] = list(summary_stats['pii_types_found'])
    
    print(f"\n📊 Overall Summary:")
    print(f"   Documents Processed: {summary_stats['total_documents']}")
    print(f"   Total OCR Regions: {summary_stats['total_ocr_regions']}")
    print(f"   Total PII Entities: {summary_stats['total_pii_entities']}")
    print(f"   PII Types Found: {', '.join(summary_stats['pii_types_found'])}")
    
    return {
        'results': all_results,
        'summary': summary_stats
    }

# Example usage with sample files
sample_files = []
if os.path.exists('samples'):
    for f in os.listdir('samples'):
        if f.lower().endswith(('.jpg', '.jpeg')):
            sample_files.append(os.path.join('samples', f))

if sample_files:
    print(f"Found {len(sample_files)} sample images:")
    for f in sample_files:
        print(f"   - {f}")
    
    # Process all samples
    batch_results = process_multiple_documents(sample_files)
    
    # Save batch results
    with open('batch_processing_results.json', 'w') as f:
        json.dump(batch_results, f, indent=2, default=str)
    
    print(f"\n💾 Batch results saved to: batch_processing_results.json")
else:
    print("No JPEG files found in samples directory")

## 5. Performance Evaluation

Evaluate the pipeline performance on different types of handwritten documents.

In [ ]:
def evaluate_pipeline_performance(results_dict):
    """
    Evaluate pipeline performance metrics
    
    Args:
        results_dict: Dictionary containing processing results
    """
    print("\n📈 Pipeline Performance Evaluation")
    print("=" * 40)
    
    if 'results' in results_dict:
        # Batch results
        results = results_dict['results']
        summary = results_dict['summary']
        
        print(f"Documents Processed: {summary['total_documents']}")
        print(f"Average OCR Regions per Document: {summary['total_ocr_regions'] / summary['total_documents']:.1f}")
        print(f"Average PII Entities per Document: {summary['total_pii_entities'] / summary['total_documents']:.1f}")
        print(f"PII Detection Rate: {(summary['total_pii_entities'] > 0) * 100:.0f}%")
        
        # Document-level analysis
        print("\n📋 Document-Level Analysis:")
        for doc_name, doc_results in results.items():
            pii_count = doc_results['summary']['pii_count']
            ocr_count = doc_results['summary']['ocr_regions']
            pii_types = ', '.join(doc_results['summary']['pii_types']) if doc_results['summary']['pii_types'] else 'None'
            
            print(f"   {doc_name}:")
            print(f"     OCR Regions: {ocr_count}")
            print(f"     PII Entities: {pii_count}")
            print(f"     PII Types: {pii_types}")
    
    print("\n✅ Evaluation complete!")

# Run evaluation if we have results
if 'batch_results' in locals():
    evaluate_pipeline_performance(batch_results)

## 6. Usage Instructions

### For New Handwritten Documents:

1. **Place your JPEG files** in the `samples/` directory
2. **Run the pipeline** using the functions above
3. **Check results** in the generated JSON files and redacted images

### Command Line Usage:
```bash
# Process a single document
python -m ocr_pipeline.cli your_document.jpg --out results.json --redact --redacted-out redacted.jpg

# Start web interface
python -m ocr_pipeline.api
# Then visit: http://127.0.0.1:5000/debug
```

### Pipeline Capabilities:
- ✅ **Handles tilted images** - Automatic deskewing
- ✅ **Different handwriting styles** - Hybrid OCR approach
- ✅ **Medical notes/forms** - Specialized PII patterns
- ✅ **Robust preprocessing** - Denoising, thresholding, morphological operations
- ✅ **Comprehensive PII detection** - Names, phones, dates, SSNs, addresses
- ✅ **Visual redaction** - Automatic bounding box overlay

### Supported PII Types:
- **PERSON** - Names and personal identifiers
- **PHONE** - Phone numbers (various formats)
- **EMAIL** - Email addresses
- **DATE** - Dates and timestamps
- **SSN** - Social Security Numbers
- **ORG** - Organizations
- **GPE** - Geographical locations